<a href="https://colab.research.google.com/github/ppbgarcia/Projects1/blob/main/GA_for_combinatorial_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SOBRE OTIMIZAÇÃO COMBINATORIAL - ENACOM**

Apesar de ser mais utilizado para problemas contínuos, para esse desafio foi escolhida a utilização de um algoritmo genético para a otimização combinatorial. Cada indivíduo é uma combinação e a ativação de cada gene a presença ou não do correspondente investimento na melhor solução. A cada geração, o melhor indivíduo é salvo e introduzido na população da próxima geração.
Não foi necessária a utilização para a solução a densidade de valor de cada investimento, pois o algoritmo usaria apenas a condição do indivíduo na função.

Abaixo, há uma descrição do algoritmo:


1.   Importar os dados dos investimentos de um arquivo (para o desafio em questão, foi-se declarado o dataframe manualmente, afim de ser executável por vocês)
2.   Verificar restrições (Há restrições 'inúteis'?)
     - Primeira restrição desabilita o investimento 5 caso 1 for habilitado
     - Segunda restrição é igual a somar ao investimento 2 o investimento 4
3.   Criar vetor inicial de booleanos
4.   Aplicar N-mutações ao vetor inicial
5.   Torneio aleatório nos vetores e seleção de parentes
6.   Geração de filhos a partir de todos pais
7.   Mutação nos filhos
8.   Verificação das restrições nos filhos
9.   Voltar ao passo 5 i-gerações




**DECLARAÇÃO DAS FUNÇÕES AUXILIARES**

In [93]:
def gera_indv(tabela, total):
  invest = 0
  i = 0
  indexes = list(bytearray(len(tabela)))
  for n in range(len(tabela)):
    if total >= (invest + tabela.Investimento[n]):
      indexes[n] = True
      invest += tabela.Investimento[n]
    if indexes[n] == False:
      tabela = tabela.drop(index = n)
  indexes = np.array(indexes,dtype = bool)
  
  return indexes

In [94]:
def limitadores(indexes, investimentos, maximo):
  if indexes[0] == True:
    indexes[4] = False
  if indexes[4] == True:
    indexes[0] = False
  if indexes[1] == True:
    indexes[3] = False
  if indexes[3] == True:
    indexes[1] = False
  
  return indexes

In [95]:
def mutacao(individuo, taxa, investimentos, maximo):
  for i in range(len(individuo)):
    aux = random.random()
    if taxa < aux:
      individuo[i] = not individuo[i]
  individuo = limite_maximo(individuo, investimentos, maximo)
  
  return np.array(individuo)

In [96]:
def limite_maximo(indexes, investimentos, maximo):
  while (investimentos.Investimento[indexes].sum()) > (maximo):
    indexes[random.randint(0,len(indexes)-1)] = False
  indexes = limitadores(indexes, investimentos, maximo)
  
  return indexes

In [97]:
def torneio(indexes, investimentos):
  aux = (len(indexes)-1)
  i = 0
  vencedores = indexes[i]
  melhor = [False]*len(investimentos)
  while i < aux:
    ind1 = indexes[i]
    ind2 = indexes[i+1]
    if (investimentos.Retorno[ind1].sum()) > (investimentos.Retorno[ind2].sum()):
      vencedores = np.vstack((vencedores, ind1))
      if (investimentos.Retorno[ind1].sum() > investimentos.Retorno[melhor].sum()):
        melhor = ind1
    else:
      if (investimentos.Retorno[ind2].sum()) > (investimentos.Retorno[melhor].sum()):
        melhor = ind2
      vencedores = np.vstack((vencedores, ind2))
    i+=2

  return vencedores, melhor


In [98]:
def cruzamento(individuos, aux1, investimentos, maximo):
  for j in range(len(individuos)):
    for i in range(len(aux1)):
      if random.random()<0.1:
        individuos[j][-i] = aux1[-i]
  
  return individuos


**DECLARAÇÃO DA FUNÇÃO PRINCIPAL E BIBLIOTECAS**

In [99]:
import pandas as pd
import numpy as np
import random 
import warnings

warnings.filterwarnings("ignore")

def GA(geracoes, pop, arquivo):
  maximo = 1000000

#------------------IMPORTAÇÃO DA TABELA DE INVESTIMENTOS-----------------------#
  investimentos = arquivo
  #investimentos = pd.read_excel(arquivo)

#-------------------SIMPLIFICAÇÃO DA PRIMEIRA REGRA ---------------------------#
  investimentos.loc[1] = investimentos.loc[1]+investimentos.loc[3]
  investimentos.Descrição[1] = 'MGL e P&D I' #Primeiro + segundo investimento

#-------------------GERAÇÃO DOS PRIMEIROS INDIVIDUOS---------------------------#

  individuos = gera_indv(investimentos, maximo)

#-------------------MUTAÇÃO PARA DA PRIMEIRA GERAÇÃO---------------------------#

  parentes = [individuos]
  for i in range(pop):
   aux = np.array(mutacao(individuos, 0.3, investimentos, maximo))
   aux = limite_maximo(aux, investimentos, maximo)
   parentes = np.vstack((aux, parentes))
  vencedores, melhor1 = torneio(parentes,investimentos)

#-----------------------------TORNEIO E MUTAÇÃO--------------------------------#
  for n in range(geracoes):
   vencedores, aux = torneio(vencedores,investimentos)
   melhor = limite_maximo(aux, investimentos, maximo)
  
   if (investimentos.Retorno[melhor].sum()) > (investimentos.Retorno[melhor1].sum()):
     melhor1 = melhor #Reservando melhor combinação
    
   for i in range(int(pop/2)):
     aux = np.array(mutacao(vencedores[i], 0.2, investimentos, maximo))
     vencedores = np.vstack((vencedores, aux))
#------------------------------CRUZAMENTO--------------------------------------#

   vencedores = cruzamento(vencedores, melhor1, investimentos, maximo)
   vencedores[0] = melhor1 #

#----------------RETORNO DA MELHOR POSIÇÃO E MELHOR SOMA-----------------------#
  return investimentos[melhor1], investimentos.Retorno[melhor1].sum()

**CHAMADA DA FUNÇÃO GA (GENETIC ALGORITHM)**

Pela característica estocástica, relativamente barata tratando-se de uso de memória e GPU e a velocidade de um algoritmo genético, optou-se por iterar a função N-vezes, de forma a garantir o melhor máximo possível. Pela dimensão relativamente reduzida, uma população baixa e poucas gerações, poucas iterações garantem uma porcentagem de acerto muito alta.

In [153]:
investimento = {
    'Descrição' : ['Ampliação da capacidade do armazém ZDP em 5%', 'Ampliação da capacidade do armazém MGL em 7%', 'Compra de empilhadeira', 'Projeto de P&D I', 'Projeto de P&D II', 'Aquisição de novos equipamentos', 'Capacitação de funcionários', 'Ampliação da estrutura de carga rodoviária'],
    'Investimento' : [470000, 400000, 170000, 270000, 340000, 230000, 50000, 440000],
    'Retorno' : [410000, 330000, 140000, 250000, 320000, 320000, 90000, 190000]
}
investimento = pd.DataFrame(investimento)

media = 0
iter = 10
gen = 50
pop = 25
melhor_soma = 0
pior_valor = 990000
melhor_individuo = []
exat = 0
soma = 0

#----------------------CHAMADA DA FUNÇÃO GA N-VEZES----------------------------#
for i in range(iter):
  melhor, soma = GA(gen, pop, investimento)
  if soma > melhor_soma:
    melhor_soma = soma
    melhor_individuo = melhor
  elif soma < pior_valor:
    pior_valor = soma
  media += soma/iter

#-------------------------------RESULTADOS-------------------------------------#


print('\t\t\t       DADOS DE 100 ITERAÇÕES')
print('\t\t\t\t   INVESTIMENTOS\n',investimento, '\n\n')
print('\t\t\t\tMELHORES INVESTIMENTOS\n',melhor_individuo, '\n\n')
print('\nMédia: ',media, '\t Melhor valor: ', melhor_soma, '\t Pior valor:', pior_valor)

			       DADOS DE 100 ITERAÇÕES
				   INVESTIMENTOS
                                       Descrição  Investimento  Retorno
0  Ampliação da capacidade do armazém ZDP em 5%        470000   410000
1                                   MGL e P&D I       3100000  2830000
2                        Compra de empilhadeira        170000   140000
3                              Projeto de P&D I        270000   250000
4                             Projeto de P&D II        340000   320000
5               Aquisição de novos equipamentos        230000   320000
6                   Capacitação de funcionários         50000    90000
7    Ampliação da estrutura de carga rodoviária        440000   190000 


				MELHORES INVESTIMENTOS
                          Descrição  Investimento  Retorno
1                      MGL e P&D I        670000   580000
5  Aquisição de novos equipamentos        230000   320000
6      Capacitação de funcionários         50000    90000 



Média:  981000.0 	 Melhor valor:  99000